In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm.notebook import tqdm
from nasbench import api
from search_spaces import load_nasbench_101
from random_search import run_random_search, random_spec

In [2]:
nasbench = load_nasbench_101()
full_metrics = nasbench.computed_statistics

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 46 seconds


In [3]:
# Each model (point) is the validation accuracy from a single training run. Test accuracies are averaged over three runs. 
accuracies = []

for _, model in tqdm(full_metrics.items()):
    metrics = model[108]
    metric = random.choice(metrics)
    val_acc = metric['final_validation_accuracy']
    test_acc = np.average([metric['final_test_accuracy'] for metric in metrics])
    
    accuracies.append([val_acc, test_acc])
accuracies = np.array(accuracies)

  0%|          | 0/423624 [00:00<?, ?it/s]

In [4]:
percent_acc = pd.DataFrame(accuracies * 100, columns=['validation', 'test'])

In [5]:
# oracle_val, oracle_test = percent_acc['validation'].max(), percent_acc['test'].max()
print(f"Best Validation Accuracy: {percent_acc['validation'].max()}")
print(f"Best Test Accuracy: {percent_acc['test'].max()}")

Best Validation Accuracy: 95.11218070983887
Best Test Accuracy: 94.3175752957662


In [6]:
percent_acc.to_csv('outputs/oracle_accuracy.csv', index=False)

In [ ]:
# Averaged over 100 oracle experiments, where in each experiment we randomly select one of 3 validation results, 
# Variance Test for Oracle
oracle_val_acc = []
oracle_test_acc = []

# Repeat oracle method for 100 times
for _ in tqdm(range(100)):
    val_acc = []
    test_acc = []
    
    for _, model in full_metrics.items():
        metrics = model[108]
        metric = random.choice(metrics)
        val_acc.append(metric['final_validation_accuracy'])
        test_acc.append(metric['final_test_accuracy'])
    
    oracle_val_acc.append(np.max(val_acc))
    oracle_test_acc.append(np.max(test_acc))

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
oracle_val_acc, oracle_val_std = np.average(oracle_val_acc) * 100, np.std(oracle_val_acc) * 100
print("Oracle Validation Accuracy")
print(f"avg: {oracle_val_acc} std: {oracle_val_std}")

oracle_test_acc, oracle_test_std = np.average(oracle_test_acc) * 100, np.std(oracle_test_acc) * 100
print("Oracle Test Accuracy")
print(f"avg: {oracle_test_acc} std: {oracle_test_std}")

In [9]:
np.save('outputs/oracle_val_acc.npy', oracle_val_acc)
np.save('outputs/oracle_val_std.npy', oracle_val_std)

np.save('outputs/oracle_test_acc.npy', oracle_test_acc)
np.save('outputs/oracle_test_std.npy', oracle_test_std)